In [8]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from onnx.helper import make_graph, make_model, set_model_props
from onnx import helper
import onnx
import numpy as np


model = ModelWrapper('tinyyolo-20210831.onnx')

def get_init(model, tensor_name):
    for init in model.graph.initializer:
        if tensor_name == init.name:
            init_val = init.raw_data
            break
    return np.frombuffer(init_val, dtype=np.int64)
    
def add_attr(model, node):
    for ind,n in enumerate(model.graph.node):
        new_attr = []
        if node in n.name:            
            new_attr.append(helper.make_attribute("starts", get_init(model,n.input[1])))
            new_attr.append(helper.make_attribute("ends", get_init(model,n.input[2])))
            new_attr.append(helper.make_attribute("axes", get_init(model,n.input[3])))
            if len(n.input[1:]) >3:
                new_attr.append(helper.make_attribute("steps", get_init(model,n.input[4])))
            for na in new_attr:
                n.attribute.append(na)
    return model

def del_nodes(model, node):
    select_nodes = []
    for n in model.graph.node:
        if node in n.name:
            select_nodes.append(n)
            del n.input[1:]
    return model

model = add_attr(model, "Slice")
model = del_nodes(model, "Slice")

model.save("tinyyolo-20210831_updated.onnx")


/workspace/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(


In [2]:
! python build.py

Building dataflow accelerator from tinyyolo-20210831_updated.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in build-tinyyolo-20210831_updated
Build log is at build-tinyyolo-20210831_updated/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 243, in step_tidy_up
    model = model.transform(FoldConstants())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/fold_constants.py", line 59, in apply
    oxe.execute_node(n, execution_context, graph)
  File "/workspace/qonnx/src/qonnx/core/onnx_exec.py", l

In [ ]:

def overwrite_opset(model, new_opset):
    """
    Overwrites the main opset in an ONNX file.
    Does not change any node definition.
    :param model: ONNX model
    :param new_opset: new opset
    :return: ONNX model
    """
    graph = make_graph(
        model.graph.node, model.graph.name, model.graph.input,
        model.graph.output, model.graph.initializer,
        sparse_initializer=model.graph.sparse_initializer)
    onnx_model = make_model(graph, functions=model.functions)
    onnx_model.ir_version = model.ir_version
    onnx_model.producer_name = model.producer_name
    onnx_model.producer_version = model.producer_version
    onnx_model.domain = model.domain
    onnx_model.model_version = model.model_version
    onnx_model.doc_string = model.doc_string
    if len(model.metadata_props) > 0:  # pragma: no cover
        values = {p.key: p.value for p in model.metadata_props}
        set_model_props(onnx_model, values)

    del onnx_model.opset_import[:]  # pylint: disable=E1101
    for oimp in model.opset_import:
        op_set = onnx_model.opset_import.add()  # pylint: disable=E1101
        op_set.domain = oimp.domain
        op_set.version = new_opset if oimp.domain == '' else oimp.version
    return onnx_model